In [2]:
import json
import os
import psycopg2
from web3 import Web3
import requests
import time
import re
import plotly.graph_objects as go
import pandas as pd
from pprint import pprint

CONN_PARAMS = {
    'dbname': 'ethereum_uncleanliness',
    'user': 'postgres',
    'password': 'admin',
    'host': "localhost",
    'port': "5432",
}

In [3]:
w3 = Web3(Web3.IPCProvider('/tmp/reth.ipc'))
w3.is_connected()

True

In [4]:
CHAIN_ID = {
    42161: 'Arbitrum',
    8453:  'Base',
    2741:  'Abstract',
    10:    'Optimism',
    137:   'Polygon',
    59144: 'Linea',
    324:   'ZKSync',
    130:   'Unichain',
    480:   'WorldChain',
    1868:  'Soneium',
    81457: 'Blast',
    1135:  'Lisk',
    56:    'BSC',
    250:   'Fantom',
    100:   'Gnosis',
    66:    'OKXChain',
    25:    'Cronos',
    43114: 'Avalanche',
    1088:  'Metis',
    7700:  'Canto',
    1101:  'Polygon',
    57073: 'Ink',
    56288: 'Boba_BNB',
    1666600000: 'Harmony',
    1313161554: 'Aurora',

    'circle_3': 'Arbitrum',
    'circle_5': 'Solana',
    'circle_6': 'Base',

    'okx_6': 'TRON',
    'okx_8453': 'Base',
    'okx_0': 'BTC',
    'okx_195': 'TRON',
    'okx_42161': 'Arbitrum',
    'okx_56': 'BSC',
    'okx_81457': 'Blast',
    'okx_784': 'Sui',
    'okx_501': 'Solana',
    'okx_137': 'Polygon',
    'okx_10': 'Optimism',
    'okx_9004': 'Starknet',
    'okx_196': 'Polygon',
    'okx_324': 'ZKSync',
    'okx_59144': 'Linea',
    'okx_534352': 'Scroll',
    
    'stargate_102':   'BSC',
    'stargate_106':   'Avalanche',
    'stargate_109':   'Polygon',
    'stargate_110':   'Arbitrum',
    'stargate_30110': 'Arbitrum',
    'stargate_111':   'Optimism',
    'stargate_30111': 'Optimism',
    'stargate_184':   'Base',
    'stargate_30184': 'Base',
    'stargate_183':   'Linea',
    'stargate_30183': 'Linea',
    'stargate_30214': 'Scroll',
    'stargate_30362': 'Berachain',
    'stargate_30284': 'Iota',
    'stargate_30324': 'Abstract',

    'mayan_1': 'Solana',
    'mayan_2': 'Solana',
    'mayan_4': 'BSC',
    'mayan_23': 'Arbitrum',

    'lifi_1151111081099710': 'Solana',
    'lifi_20000000000001': 'Bitcoin',

    'wormhole_0': 'Ethereum',
    'wormhole_1': 'Solana',
    'wormhole_4': 'BSC',
    'wormhole_32': 'Sei',
    'wormhole_30': 'Base',
    'wormhole_21': 'Sui',
    'wormhole_22': 'Aptos',
    'wormhole_5': 'Polygon',
    'wormhole_16': 'Moonbeam',
    'wormhole_14': 'Celo',
    'wormhole_24': 'Optimism',
    'wormhole_23': 'Arbitrum',
    'wormhole_8': 'Algorand',

    'debridge_7565164': 'Solana',
    'debridge_42161': 'Arbitrum',
    'debridge_56': 'BSC',
    'debridge_8453': 'Base',
    'debridge_10': 'Optimism',
    'debridge_137': 'Polygon',
    'debridge_43114': 'Avalanche',
    'debridge_100000002': 'Gnosis',
    'debridge_100000004': 'Metis',

    'swft_TRON': 'TRON',
    'swft_BSC':  'BSC',
    'swft_TRX':  'TRON',
    'swft_ARB':  'Arbitrum',
    'swft_MATIC': 'Polygon',
    'swft_ERC20': 'Ethereum',
    'swft_SOL': 'Solana',
    'swft_C-Chain': 'Avalanche',
    'swft_OKT': 'OKXChain',
    'swft_LINK': 'Ethereum',
    'swft_MNT': 'Mantle',
    'swft_Optimism': 'Optimism',
    'swft_ZKSYNC': 'ZKSync',
    'swft_BASE': 'Base',
    'swft_TON': 'TON',
    'swft_SUI': 'Sui',
    'swft_OKExChain': 'OKXChain',
    'swft_SCROLL': 'Scroll',

    'transit_95500': 'TRON',
    'transit_0': 'Ethereum',
    'transit_56': 'BSC',
    'transit_42161': 'Arbitrum',
    'transit_728126428': 'TRON',
    'transit_10': 'Optimism',
    'transit_137': 'Polygon',
    'transit_128': 'HECO',
    'transit_8453': 'Base',
    'transit_250': 'Fantom',
    'transit_324': 'ZKSync',
    'transit_185': 'Mint'
}

In [5]:
def extract_transactions(to_address):
    to_address = w3.to_checksum_address(to_address)
    with psycopg2.connect(**CONN_PARAMS) as conn:
        with conn.cursor() as curs:
            transactions = []
            sql_select = \
            """
            SELECT * FROM related_transactions_final WHERE to_address = '{}' ORDER BY block_number ASC; 
            """.format(to_address)
            curs.execute(sql_select)
            rows = curs.fetchall()
            for row in rows:
                transactions.append(row)
            return transactions

In [6]:
def extract_all_transactions(to_address_set):
    all_transactions = []
    for to_address in to_address_set:
        all_transactions.extend(extract_transactions(to_address))
    print(len(all_transactions))
    print('='*50)
    return all_transactions

In [7]:
def topic_summary(all_transactions):
    topics = {}
    for transaction in all_transactions:
        transaction_hash = transaction[3]
        transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
        if transaction_receipt:
            for log in transaction_receipt.logs:
                if len(log.topics) == 0:
                    continue
                target_topic = '0x' + log.topics[0].hex()
                if target_topic not in topics:
                    topics[target_topic] = 1
                else:
                    topics[target_topic] += 1

    sum_topics = sum(topics.values())
    print(sum_topics)
    print('='*50)
    # Sort topics by count
    topics = dict(sorted(topics.items(), key=lambda item: item[1], reverse=True))
    for topic, count in topics.items():
        print(topic, count)
    print('='*50)

### THORChain

In [6]:
thorchain_addresses = [
    '0xD37BbE5744D730a1d98d8DC97c42F0Ca46aD7146',
]
thorchain_transactions = extract_all_transactions(thorchain_addresses)
topic_summary(thorchain_transactions)

61150
77347
0xa9cd03aa3c1b4515114539cd53d22085129d495cb9e9f9af77864526240f1bf7 46514
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 15351
0x05b90458f953d3fcb2d7fb25616a2fddeca749d0c47cc5c9832d0266b5346eea 11070
0xef519b7eb82aaf6ac376a6df2d793843ebfd593de5f1a0601d3cc6ab49ebb395 3342
0x8e5841bcd195b858d53b38bcf91b38d47f3bc800469b6812d35451ab619c6f6c 224
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 206
0xdec2bacdd2f05b59de34da9b523dff8be42e5e38e818c82fdb0bae774387a724 142
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 107
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67 105
0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1 101
0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822 101
0x06b541ddaa720db2b10a4d0cdac39b8d360425fc073085fac19bc82614677987 51
0xa0a19463ee116110c9b282012d9b65cc5522dc38a9520340cbaf3142e550127f 16
0xb46eeb115d9875d9d717a1b1dd704bc9a45fdeb55eecc9c6c4f777ffa5

In [7]:
thorchain_transactions_filtered = {}
for transaction in thorchain_transactions:
        transaction_hash = transaction[3]
        transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
        if transaction_receipt:
            for log in transaction_receipt.logs:
                target_topic = '0x' + log.topics[0].hex()

                if target_topic == '0xef519b7eb82aaf6ac376a6df2d793843ebfd593de5f1a0601d3cc6ab49ebb395': # Deposit (from ETH)
                    memo = bytes.fromhex(log.data.hex()[128:]).decode('ascii') 
                    thorchain_transactions_filtered[transaction_hash] = [memo, transaction]

print(len(thorchain_transactions_filtered))
print('='*50)

3342


In [8]:
chains = {}
for transaction_hash, (memo, transaction) in thorchain_transactions_filtered.items():
    memo = memo.replace('\x00', '')
    if memo:
        chain_token = memo[1:].split(':')[1]
        if chain_token not in chains:
            chains[chain_token] = 1
        else:   
            chains[chain_token] += 1
chains = dict(sorted(chains.items(), key=lambda item: item[1], reverse=True))
for chain_token, count in chains.items():
    print(chain_token, count)

BTC.BTC 2231
b 631
LTC.LTC 236
AVAX.AVAX 37
s 33
BSC.BNB 31
THOR.RUNE 30
BNB.BNB 11
BCH.BCH 11
ETH.ETH 11
r 10
ETH.WBTC 9
AVAX.USDT-0X9702230A8EA53601F5CD2DC00FDBC13D4DF4A8C7 9
DOGE.DOGE 6
GAIA.ATOM 6
l 6
BSC.USDT-0X55D398326F99059FF775485246999027B3197955 6
ETH.USDT-0XDAC17F958D2EE523A2206206994597C13D831EC7 5
BSC.USDC 4
c 2
a 2
ETH/ETH 2
ETH.USDC-06EB48 2
ETH.THOR-0XA5F2211B9B8170F694421F2046281775E8468044 2
RUNE 1
BSC.USDC-0X8AC76A51CC950D9822D68B83FE1AD97B32CD580D 1
ETH.USDC 1
DASH.DASH 1
e 1
ETH.WBTC-0X2260FAC5E5542A773AA44FBCFEDF7C193BC2C599 1
g 1


In [11]:
summary = {}
btc_addresses = {}
total_value = 0
total_dirty_value = 0

for transaction_hash, [memo, transaction] in thorchain_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    memo = memo.replace('\x00', '')
    if memo:
        chain_token = memo[1:].split(':')[1]

        # BTC
        if chain_token.split('.')[0] == 'BTC' or chain_token == 'b':
            btc_address = memo[1:].split(':')[2]
            value = transaction[8]
            dirty_value= transaction[9]
            if 'BTC' not in summary:
                summary['BTC'] = (1, value, dirty_value)
            else:
                summary['BTC'] = (summary['BTC'][0] + 1, summary['BTC'][1] + value, summary['BTC'][2] + dirty_value)
            if btc_address not in btc_addresses:
                btc_addresses[btc_address] = value
            else:
                btc_addresses[btc_address] += value

        # LTC
        if chain_token.split('.')[0] == 'LTC' or chain_token == 'l':
            value = transaction[8]
            dirty_value= transaction[9]
            if 'LTC' not in summary:
                summary['LTC'] = (1, value, dirty_value)
            else:
                summary['LTC'] = (summary['LTC'][0] + 1, summary['LTC'][1] + value, summary['LTC'][2] + dirty_value)

        # AVAX
        if chain_token.split('.')[0] == 'AVAX' or chain_token == 'a':
            value = transaction[8]
            dirty_value= transaction[9]
            if 'AVAX' not in summary:
                summary['AVAX'] = (1, value, dirty_value)
            else:
                summary['AVAX'] = (summary['AVAX'][0] + 1, summary['AVAX'][1] + value, summary['AVAX'][2] + dirty_value)

        # THOR
        if chain_token.split('.')[0] == 'THOR' or chain_token == 'r' or chain_token == 'RUNE':
            value = transaction[8]
            dirty_value= transaction[9]
            if 'THOR' not in summary:
                summary['THOR'] = (1, value, dirty_value)
            else:
                summary['THOR'] = (summary['THOR'][0] + 1, summary['THOR'][1] + value, summary['THOR'][2] + dirty_value)

        # BSC
        if chain_token.split('.')[0] == 'BSC' or chain_token == 's' or chain_token.split('.')[0] == 'BNB':
            value = transaction[8]
            dirty_value= transaction[9]
            if 'BSC' not in summary:
                summary['BSC'] = (1, value, dirty_value)
            else:
                summary['BSC'] = (summary['BSC'][0] + 1, summary['BSC'][1] + value, summary['BSC'][2] + dirty_value)

        # ETH
        if chain_token.split('.')[0] == 'ETH':
            value = transaction[8]
            dirty_value= transaction[9]
            if 'ETH' not in summary:
                summary['ETH'] = (1, value, dirty_value)
            else:
                summary['ETH'] = (summary['ETH'][0] + 1, summary['ETH'][1] + value, summary['ETH'][2] + dirty_value)

        # BCH
        if chain_token.split('.')[0] == 'BCH' or chain_token == 'c':
            value = transaction[8]
            dirty_value= transaction[9]
            if 'BCH' not in summary:
                summary['BCH'] = (1, value, dirty_value)
            else:
                summary['BCH'] = (summary['BCH'][0] + 1, summary['BCH'][1] + value, summary['BCH'][2] + dirty_value)

        # GAIA
        if chain_token.split('.')[0] == 'GAIA':
            value = transaction[8]
            dirty_value= transaction[9]
            if 'GAIA' not in summary:
                summary['GAIA'] = (1, value, dirty_value)
            else:
                summary['GAIA'] = (summary['GAIA'][0] + 1, summary['GAIA'][1] + value, summary['GAIA'][2] + dirty_value)

        # DOGE
        if chain_token.split('.')[0] == 'DOGE':
            value = transaction[8]
            dirty_value= transaction[9]
            if 'DOGE' not in summary:
                summary['DOGE'] = (1, value, dirty_value)
            else:
                summary['DOGE'] = (summary['DOGE'][0] + 1, summary['DOGE'][1] + value, summary['DOGE'][2] + dirty_value)

        # DASH
        if chain_token.split('.')[0] == 'DASH':
            value = transaction[8]
            dirty_value= transaction[9]
            if 'DASH' not in summary:
                summary['DASH'] = (1, value, dirty_value)
            else:
                summary['DASH'] = (summary['DASH'][0] + 1, summary['DASH'][1] + value, summary['DASH'][2] + dirty_value)

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<6} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    print(f"{chain:<6} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")
print('='*50)

Total Value      :      127960.806887
Total Dirty Value:      123864.882159
Percentage       :          96.799079
Chain   Count              Value        Dirty Value   Percentage
BTC      2862      113791.390356      112176.348190    98.580699
LTC       242       11252.898944        8802.519171    78.224458
AVAX       48         909.230511         908.302238    99.897906
BSC        86         709.541622         707.002416    99.642134
THOR       41         646.519614         638.830379    98.810673
ETH        31         162.045685         143.534362    88.576479
GAIA        6          33.659145          32.930606    97.835540
BCH        13          29.491809          29.481235    99.964148
DOGE        6           0.478600           0.404107    84.435252
DASH        1           0.049700           0.049700    99.999861


In [22]:

btc_addresses = dict(sorted(btc_addresses.items(), key=lambda item: item[1], reverse=True))
for btc_address, amount in btc_addresses.items():
    print(f'{btc_address:48} receives {int(amount)/1e18:18.6f} ETH')

bc1qywjt9hmlf550lm56ytusw6468gjpfpaeq0pdhn       receives       10242.000000 ETH
bc1qxyx08avsewclgqzv6kqawvk02lu4wjpv3rk8h5       receives        4413.900000 ETH
bc1qzccryvnxp9mz54mvtatk0hc0zfp095g73kgh29       receives        2984.800000 ETH
3P6oGXcUofZqK9auhn47FogdEeV4amQKy3               receives        2692.700000 ETH
bc1qf3dxu7ca5fek33mxxezpuyvmz2mhk9rcryd8qk       receives        2558.000000 ETH
bc1qwmg2cphe9xc4lysz2f98gpzksuywxcr5t02jwr       receives        2468.470000 ETH
bc1qjzzdhu23lag08syexq7zl2ujljfez928zr2n0s       receives        2278.800000 ETH
bc1qtmdr06cz9gp2yjfltwwhtpzyg7t5lfn4m6vuwh       receives        2248.300000 ETH
bc1qwh4620jzxrj3lm082nksdpelaryrpr890chsv9       receives        1947.000000 ETH
bc1q60r42pzf5mqkll8ht39ycum6ayl7e5469vhsfv       receives        1741.400000 ETH
bc1qna0xnztutc5um4zhanht0dtw3l5eqafuyk559n       receives        1677.700000 ETH
bc1qrtyp4stpsd69j973ft5uek6d9xvc36q92lxmex       receives        1613.700000 ETH
bc1q8xw9a06jkj264pqcgvqp5s5y

### Arbitrum

In [114]:
arbitrum_addresses = [
    '0x4Dbd4fc535Ac27206064B68FfCf827b0A60BAB3f',
]
arbitrum_transactions = extract_all_transactions(arbitrum_addresses)
topic_summary(arbitrum_transactions)

1673
3346
0xff64905f73a67fb594e0f940a8075a860db489ad991e032f48c81123eb52d60b 1673
0x5e3c1311ea442664e8b1611bfabef659120ea7a0a2cfc0667700bebc69cbffe1 1661
0x23be8e12e420b5da9fb98d8102572f640fb3c11a0085060472dfc0ed194b3cf7 12


In [116]:
total_value = 0
total_dirty_value = 0
for transaction in arbitrum_transactions:
    total_value += transaction[8]
    total_dirty_value += transaction[9]

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
print(f"{'Arbitrum':<10} {len(arbitrum_transactions):6} {int(total_value)/1e18:18.6f} {int(total_dirty_value)/1e18:18.6f} {total_dirty_value/total_value*100:12.6f}")

Total Value      :       62861.450703
Total Dirty Value:       62610.416893
Percentage       :          99.600655
Chain       Count              Value        Dirty Value   Percentage
Arbitrum     1673       62861.450703       62610.416893    99.600655


### OKX

In [416]:
okx_addresses = [
    '0xFc99f58A8974A4bc36e60E2d490Bb8D72899ee9f',
]
okx_transactions = extract_all_transactions(okx_addresses)
topic_summary(okx_transactions)

1974
21272
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 7355
0x6ea4114cf93cff6c9ff22aef8d246e78a0368fa48033adc49edde5fa273f69a1 1974
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 1373
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67 1369
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 1035
0xf6481cbc1da19356c5cb6b884be507da735b89f21dc4bbb7c9b7cc0968b03b7a 996
0xb9dae57db52a734b183c77227c96068231beb6a93a060ca7a9d3164f716714ea 978
0x7724394874fdd8ad13292ec739b441f85c6559f10dc4141b8d4c0fa4cbf55bdb 974
0x1bb43f2da90e35f7b0cf38521ca95a49e68eb42fac49924930a5bd73cdf7576c 974
0x45f377f845e1cc76ae2c08f990e15d58bcb732db46f92a4852b956580c3a162f 776
0xffc60ee157a42f4d8edbd1897e6581a96d9ed04e44fb2ab53a47ce1eb8f2775b 429
0x85496b760a4b7f8d66384b9df21b381f5d1b1e79f229a47aaf4c232edc2fe59a 378
0x15955c5a4cc61b8fbb05301bce47fd31c0e6f935e1ab97fdac9b134c887bb074 368
0xef519b7eb82aaf6ac376a6df2d793843ebfd593de5f1a0601d3cc6ab49e

In [ ]:
adaptor_ids = {}
to_chain_dict = {}
okx_transactions_filtered = {}
for transaction in okx_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0xf6481cbc1da19356c5cb6b884be507da735b89f21dc4bbb7c9b7cc0968b03b7a' or \
                target_topic == '0xb9dae57db52a734b183c77227c96068231beb6a93a060ca7a9d3164f716714ea':
                
                adaptor_id = int('0x' + log.topics[1].hex(), 16)
                if adaptor_id not in adaptor_ids:
                    adaptor_ids[adaptor_id] = 1
                else:
                    adaptor_ids[adaptor_id] += 1

                to_chain_id = int(log.data.hex()[64*8:64*9], 16)
                to_chain_id = 'okx_' + str(to_chain_id)
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                okx_transactions_filtered[transaction_hash] = [to_chain, transaction]
                
        # if adaptor_id == 3:  # SWFT Swap
        # if adaptor_id == 39: # Stargate: Pool Native
        # if adaptor_id == 42: # Stargate: Pool Native
        # if adaptor_id == 9:  # Stargate Finance: S*USDT Token
        # if adaptor_id == 23: # THORChain
        # if adaptor_id == 18: # Circle: Token Messenger
        # if adaptor_id == 30: # Wormhole: Ethereum Core Bridge
        # if adaptor_id == 21: # Wormhole: Ethereum Core Bridge
        # if adaptor_id == 46: # Wormhole: Ethereum Core Bridge
        # if adaptor_id == 17: # Wanchain: CrossProxy
        # if adaptor_id == 36: # Across Protocol: Ethereum Spoke Pool V2
        # if adaptor_id == 52: # Across Protocol: Ethereum Spoke Pool V2
        # if adaptor_id == 10: # Across Protocol: Ethereum Spoke Pool V2
        # if adaptor_id == 6:  # Arbitrum: Delayed Inbox
        # if adaptor_id == 2:  # Celer Network: cBridge V2
        # if adaptor_id == 43: # Sui: Sui Bridge
        # if adaptor_id == 37: # Socket: GasMovr
        # if adaptor_id == 13: # Biconomy: Liquidity Pool
        # if adaptor_id == 7:  # Optimism: Gateway
        # if adaptor_id == 38: # Base: Base Bridge
        # if adaptor_id == 47: # Mayan Finance: Mayan Forwarder
        # if adaptor_id == 5:  # Polygon (Matic): Ether Bridge
        # if adaptor_id == 25: # Starknet: StarkGate ETH Bridge
        # if adaptor_id == 41: # Allbridge: Core Bridge
        # if adaptor_id == 34: # deBridge: DlnSource
                
                
# print('='*50)
# adaptor_ids = dict(sorted(adaptor_ids.items(), key=lambda item: item[1], reverse=True))
# for adaptor_id, count in adaptor_ids.items():
#     print(adaptor_id, count)

print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)

okx_6 978
okx_8453 397
okx_0 273
okx_195 114
okx_42161 74
okx_56 48
okx_81457 31
okx_784 24
okx_501 11
okx_137 9
okx_10 6
okx_9004 3
okx_196 2
okx_324 2
okx_59144 1
okx_534352 1


In [527]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in okx_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :       20815.404014
Total Dirty Value:       20426.103420
Percentage       :          98.129748
Chain       Count              Value        Dirty Value   Percentage
TRON         1092       11568.857712       11444.552409    98.925518
BTC           273        4764.790798        4698.071371    98.599741
Arbitrum       74        1863.980766        1771.070669    95.015501
Base          397        1003.886073         910.239124    90.671556
Optimism        6         511.879340         511.879340   100.000000
Sui            24         456.511293         456.146404    99.920070
Polygon        11         316.220681         316.200305    99.993556
Solana         11         234.117100         234.116773    99.999860
BSC            48          85.945028          77.286655    89.925684
Starknet        3           2.881848           2.703688    93.817864
Blast          31           4.982193           2.547872    51.139560
ZKSync          2           1.285143           1.285143   

In [507]:
data = '000000000000000000000000d60121b9ba42b12c4c08682ff5ad06c4032b1d37000000000000000000000000d60121b9ba42b12c4c08682ff5ad06c4032b1d37000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee00000000000000000000000000000000000000000000000115808d2b0e13c3f0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000006000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000021050000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000030786436303132316239626134326231326334633038363832666635616430366334303332623164333700000000000000000000000000000000000000000000'
for i in range(0, len(data), 64):
    print(data[i:i+64])
    print(int(data[i:i+64], 16))

000000000000000000000000d60121b9ba42b12c4c08682ff5ad06c4032b1d37
1221749263606756509575517077000706607622522608951
000000000000000000000000d60121b9ba42b12c4c08682ff5ad06c4032b1d37
1221749263606756509575517077000706607622522608951
000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
1364068194842176056990105843868530818345537040110
00000000000000000000000000000000000000000000000115808d2b0e13c3f0
19996137561584288752
0000000000000000000000000000000000000000000000000000000000000000
0
00000000000000000000000000000000000000000000000000000000000000c0
192
0000000000000000000000000000000000000000000000000000000000000006
6
0000000000000000000000000000000000000000000000000000000000000000
0
0000000000000000000000000000000000000000000000000000000000002105
8453
0000000000000000000000000000000000000000000000000000000000000000
0
0000000000000000000000000000000000000000000000000000000000000000
0
3078643630313231623962613432623132633463303836383266663561643036
219237300153172199317894248274

### Across Protocol

In [61]:
across_addresses = [
    '0xB4A8d45647445EA9FC3E1058096142390683dBC2', # Across Protocol: SpokePoolVerifer
    '0x5c7BCd6E7De5423a257D81B442095A1a6ced35C5', # Across Protocol: Ethereum Spoke Pool V2
    '0x269727F088F16E1Aea52Cf5a97B1CD41DAA3f02D', # Across Protocol: SpokePoolVerifer
    '0xc186fA914353c44b2E33eBE05f21846F1048bEda', # Across Protocol: Hub Pool V2
    '0x4D9079Bb4165aeb4084c526a32695dCfd2F77381', # Across Protocol: Old Ethereum Spoke Pool V2
    '0xbcfEb7d29BB1b40c6eA3A9e3d0822a79819d63F3', # Across Protocol: Ethereum Spoke Pool V2
]
across_transactions = extract_all_transactions(across_addresses)
topic_summary(across_transactions)

1225
2474
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 1156
0xa123dc29aebf7d0c3322c8eeb5b999e859f39937950ed31056532713d0de396f 828
0xafc4df6845a4ab948b492800d3d8a25d538a102a2bc07cd01f1cfa097fddcff6 255
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 91
0x32ed1a409ef04c7b0227189c3a103dc5ac10e775a15b785dcc510201f7c25ad3 76
0x4a4fc49abd237bfd7f4ac82d6c7a284c69daaea5154430cff04ad7482c6c4254 42
0x3c69701a61c79a92ef9692903aaa0068bce8771361ecb09547391e4fb4df8537 17
0xcda1185f28599e6bd14ab8a68b3c30a11e1dce4256b5e67e94dd3fd846a6c589 7
0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65 2


In [108]:
to_chain_dict = {}
across_transactions_filtered = {}
for transaction in across_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0xa123dc29aebf7d0c3322c8eeb5b999e859f39937950ed31056532713d0de396f' or \
                target_topic == '0xafc4df6845a4ab948b492800d3d8a25d538a102a2bc07cd01f1cfa097fddcff6' or \
                target_topic == '0x32ed1a409ef04c7b0227189c3a103dc5ac10e775a15b785dcc510201f7c25ad3':
                to_chain_id = int(log.topics[1].hex(), 16)
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                across_transactions_filtered[transaction_hash] = [to_chain, transaction]

            if target_topic == '0x4a4fc49abd237bfd7f4ac82d6c7a284c69daaea5154430cff04ad7482c6c4254':
                to_chain_id = int(log.data.hex()[128:128+64], 16)
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                across_transactions_filtered[transaction_hash] = [to_chain, transaction]
            

print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

42161 740
8453 222
10 118
137 66
59144 35
324 13
130 3
480 1
1868 1
81457 1
1135 1
1201


In [97]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in across_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :       26586.499475
Total Dirty Value:       26155.004371
Percentage       :          98.377014
Chain       Count              Value        Dirty Value   Percentage
Arbitrum      740       17601.344420       17456.057804    99.174571
Optimism      118        3901.633054        3894.022346    99.804935
Base          222        3125.205505        2851.950005    91.256399
Linea          35        1376.640505        1376.538517    99.992592
Polygon        66         346.511003         341.376623    98.518264
ZKSync         13         234.926074         234.917929    99.996533
Blast           1           0.100000           0.067507    67.507360
WorldChain      1           0.050000           0.050000   100.000000
Soneium         1           0.050000           0.011207    22.414525
Unichain        3           0.030914           0.010179    32.925590
Lisk            1           0.008000           0.002253    28.163487


### Stargate

In [169]:
stargate_addresses = [
    '0x77b2043768d28E9C9aB44E1aBfC95944bcE57931', # Stargate: Pool Native
    '0x150f94B44927F078737562f0fcF3C95c01Cc2376', # Stargate Finance: ETH Router 2
]
stargate_transactions = extract_all_transactions(stargate_addresses)
topic_summary(stargate_transactions)

853
5704
0x85496b760a4b7f8d66384b9df21b381f5d1b1e79f229a47aaf4c232edc2fe59a 432
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 428
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 418
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 418
0x34660fc8af304464529f48a778e03d03e4d34bcd5f9b6f0cfbf3cd238c642f7f 410
0x6939f93e3f21cf1362eb17155b740277de5687dae9a83a85909fd71da95944e7 410
0xdf21c415b78ed2552cc9971249e32a053abce6087a0ae0fbf3f78db5174a3493 410
0xb0c632f55f1e1b3b2c3d82f41ee4716bb4c00f0f5d84cdafc141581bb8757a4f 410
0xe9bded5f24a4168e4f3bf44e00298c993b22376aad8c58c7dda9718a54cbea82 410
0x8d3ee0df6a4b7e82a7f20a763f1c6826e6176323e655af64f32318827d2112d4 410
0x61ed099e74a97a1d7f8bb0952a88ca8b7b8ebd00c126ea04671f92a81213318a 348
0x07ea52d82345d6e838192107d8fd7123d9c2ec8e916cd0aad13fd2b60db24644 348
0x1ab700d4ced0c005b164c0f789fd09fcbb0156d4c2041b8a3bfbcd961cd1567f 348
0x87e46b0a6199bc734632187269a103c05714ee0adae5b28f30723955724f37ef 2

In [173]:
to_chain_dict = {}
stargate_transactions_filtered = {}
for transaction in stargate_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0x85496b760a4b7f8d66384b9df21b381f5d1b1e79f229a47aaf4c232edc2fe59a': # Stargate: Pool Native
                to_chain_id = 'stargate_' + str(int('0x' + log.data.hex()[:64], 16))
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                stargate_transactions_filtered[transaction_hash] = [to_chain, transaction]
            if target_topic == '0x34660fc8af304464529f48a778e03d03e4d34bcd5f9b6f0cfbf3cd238c642f7f': # Swap
                to_chain_id = 'stargate_' + str(int('0x' + log.data.hex()[:64], 16))
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                stargate_transactions_filtered[transaction_hash] = [to_chain, transaction]

print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

stargate_110 293
stargate_30110 265
stargate_30111 92
stargate_111 61
stargate_30184 55
stargate_184 47
stargate_30183 13
stargate_183 9
stargate_30214 3
stargate_30362 2
stargate_30284 1
stargate_30324 1
842


In [174]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in stargate_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :       26833.374204
Total Dirty Value:       26425.021390
Percentage       :          98.478191
Chain       Count              Value        Dirty Value   Percentage
Arbitrum      558       18333.430680       18015.768350    98.267306
Optimism      153        5512.974404        5481.918716    99.436680
Base          102        2115.383779        2075.592858    98.118974
Linea          22         862.626884         850.496897    98.593831
Berachain       2           7.892157           0.619076     7.844192
Scroll          3           0.536235           0.536235   100.000000
Abstract        1           0.500061           0.059255    11.849575
Iota            1           0.030004           0.030004   100.000000


### Optimism

In [117]:
optimism_addresses = [
    '0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1',
]
optimism_transactions = extract_all_transactions(optimism_addresses)
topic_summary(optimism_transactions)

289
1386
0xcb0f7ffd78f9aee47a248fae8db181db6eee833039123e026dcbff529522e52a 289
0x35d79ab81f2b2017e19afb5c5571778877782d7a8786f5907f93b0f4702f4f23 279
0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32 254
0x8ebb2ec2465bdb2a06a66fc37a0963af8a2a6a1479d81d56fdb8cbb98096d546 254
0x2849b43074093a05396b6f2a937dee8565b15a48a7b3d4bffb732a5017380af5 250
0x4b388aecf9fa6cc92253704e5975a6129a4f735bdbd99567df4ed0094ee4ceb5 35
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 10
0x718594027abd4eaed59f95162563e0cc6d0e8d5b86b1c7be8b1b0ac3343d0396 10
0x7ff126db8024424bbfd9826e8ab82ff59136289ea440b04b39a0df1b03b9cabf 4
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 1


In [119]:
total_value = 0
total_dirty_value = 0
for transaction in optimism_transactions:
    total_value += transaction[8]
    total_dirty_value += transaction[9]

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
print(f"{'Optimism':<10} {len(optimism_transactions):6} {int(total_value)/1e18:18.6f} {int(total_dirty_value)/1e18:18.6f} {total_dirty_value/total_value*100:12.6f}")

Total Value      :       12556.070058
Total Dirty Value:       12536.994640
Percentage       :          99.848078
Chain       Count              Value        Dirty Value   Percentage
Optimism      289       12556.070058       12536.994640    99.848078


### Mayan Finance

In [228]:
mayan_addresses = [
    '0x0654874eb7F59C6f5b39931FC45dC45337c967c3', # Mayan Finance: MayanForwader
    '0xe3985E6b61b814F7Cdb188766562ba71b446B46d', # Mayan Protocol: MayaChain Router
]
mayan_transactions = extract_all_transactions(mayan_addresses)
topic_summary(mayan_transactions)

1328
4631
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 1259
0x6eb224fb001ed210e379b335e35efe88672a8ce935d981a6896b27ffdf52a3b2 606
0xa9cd03aa3c1b4515114539cd53d22085129d495cb9e9f9af77864526240f1bf7 585
0xb8543d214cab9591941648db8d40126a163bfd0db4a865678320b921e1398043 493
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 424
0x918554b6bd6e2895ce6553de5de0e1a69db5289aa0e4fe193a0dcd1f14347477 284
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67 146
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 112
0xcc16f5dbb4873280815c1ee09dbd06736cffcc184412cf7a71a0fdb75d397ca5 84
0x8c5261668696ce22758910d05bab8f186d6eb247ceac2af2e82c7dc17669b036 84
0x2fa9ca894982930190727e75500a97d8dc500233a5065e0f3126c48fbe0343c0 84
0x05b90458f953d3fcb2d7fb25616a2fddeca749d0c47cc5c9832d0266b5346eea 80
0x7cbff921ae1f3ea71284120d2aabde13587df067f2bb5c831ea6e35d7a9242ac 65
0x23278f58875126c795a4072b98b5851fe9b21cea19895b02a6224fefbb1e3298 38
0

In [230]:
to_chain_dict = {}
mayan_transactions_filtered = {}
for transaction in mayan_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0xb8543d214cab9591941648db8d40126a163bfd0db4a865678320b921e1398043': # ForwardEth
                to_chain_id = 'mayan_' + str(int(log.data.hex()[64*15:64*15+8], 16))
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                mayan_transactions_filtered[transaction_hash] = [to_chain, transaction]

print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

mayan_2 354
mayan_1 131
mayan_23 7
mayan_4 1
493


In [231]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in mayan_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :       10985.705321
Total Dirty Value:        9631.302079
Percentage       :          87.671222
Chain       Count              Value        Dirty Value   Percentage
Solana        485       10917.405321        9566.949437    87.630249
Arbitrum        7          56.000000          56.000000   100.000000
BSC             1          12.300000           8.352641    67.907654


### Multichain

In [167]:
multichain_addresses = [
    '0xBa8Da9dcF11B50B03fd5284f164Ef5cdEF910705', # Multichain: Router V6
]
multichain_transactions = extract_all_transactions(multichain_addresses)
topic_summary(multichain_transactions)

210
630
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 210
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 210
0x97116cf6cd4f6412bb47914d6db18da9e16ab2142f543b86e207c24fbd16b23a 210


In [175]:
to_chain_dict = {}
multichain_transactions_filtered = {}
for transaction in multichain_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0x97116cf6cd4f6412bb47914d6db18da9e16ab2142f543b86e207c24fbd16b23a': # LogAnySwapOut
                to_chain_id = int('0x' + log.data.hex()[128:], 16)
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                multichain_transactions_filtered[transaction_hash] = [to_chain, transaction]

print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

56 98
42161 41
137 30
10 26
250 11
66 3
25 1
210


In [176]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in multichain_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :       10930.260711
Total Dirty Value:       10801.729774
Percentage       :          98.824082
Chain       Count              Value        Dirty Value   Percentage
Polygon        30        4413.581405        4413.580886    99.999988
BSC            98        3799.458772        3679.143381    96.833354
Optimism       26        1407.189989        1407.159880    99.997860
Arbitrum       41        1120.069132        1119.727000    99.969454
OKXChain        3         109.426469         109.426469   100.000000
Fantom         11          79.536943          71.694157    90.139443
Cronos          1           0.998000           0.998000   100.000000


### ZKSync

In [197]:
zksync_addresses = [
    '0x303a465B659cBB0ab36eE643eA362c509EEb5213', # zkSync: Bridgehub (Proxy)
    '0x32400084C286CF3E17e7B677ea9583e60a000324', # zkSync Era: Diamond Proxy
    '0xaBEA9132b05A70803a4E85094fD0e1800777fBEF', # zkSync
]
zksync_transactions = extract_all_transactions(zksync_addresses)
topic_summary(zksync_transactions)

1101
1900
0x4531cd5795773d7101c17bdeb9f5ab7f47d7056017506f937083be5d6e77a382 1000
0x249bc8a55d0c4a0034b9aaa6be739bec2d4466e5d859bec9566a8553c405c838 703
0xd0943372c08b438a88d4b39d77216901079eda9ca59d45349841c099083b6830 91
0x8f5f51448394699ad6a3b80cdadf4ec68c5d724c8c3fea09bea55b3c2d0e2dd0 91
0x26464d64ddb13f6d187de632d165bd1065382ec0b66c25c648957116e7bc25c8 7
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 4
0x05518b128f0a9b11ddddebd5211a7fc2f4a689dab3a3e258d93eb13049983c3e 3
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 1


In [198]:
total_value = 0
total_dirty_value = 0
for transaction in zksync_transactions:
    total_value += transaction[8]
    total_dirty_value += transaction[9]

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
print(f"{'ZKSync':<10} {len(zksync_transactions):6} {int(total_value)/1e18:18.6f} {int(total_dirty_value)/1e18:18.6f} {total_dirty_value/total_value*100:12.6f}")

Total Value      :       12184.030466
Total Dirty Value:       12006.504170
Percentage       :          98.542959
Chain       Count              Value        Dirty Value   Percentage
ZKSync       1101       12184.030466       12006.504170    98.542959


### Base

In [199]:
base_addresses = [
    '0x3154Cf16ccdb4C6d922629664174b904d80F2C35', # Base: Base Bridge
]
base_transactions = extract_all_transactions(base_addresses)
topic_summary(base_transactions)

202
1022
0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32 202
0xcb0f7ffd78f9aee47a248fae8db181db6eee833039123e026dcbff529522e52a 202
0x8ebb2ec2465bdb2a06a66fc37a0963af8a2a6a1479d81d56fdb8cbb98096d546 202
0x35d79ab81f2b2017e19afb5c5571778877782d7a8786f5907f93b0f4702f4f23 194
0x2849b43074093a05396b6f2a937dee8565b15a48a7b3d4bffb732a5017380af5 194
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 9
0x718594027abd4eaed59f95162563e0cc6d0e8d5b86b1c7be8b1b0ac3343d0396 8
0x7ff126db8024424bbfd9826e8ab82ff59136289ea440b04b39a0df1b03b9cabf 8
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 3


In [201]:
total_value = 0
total_dirty_value = 0
for transaction in base_transactions:
    total_value += transaction[8]
    total_dirty_value += transaction[9]

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
print(f"{'Base':<10} {len(base_transactions):6} {int(total_value)/1e18:18.6f} {int(total_dirty_value)/1e18:18.6f} {total_dirty_value/total_value*100:12.6f}")

Total Value      :        7831.276427
Total Dirty Value:        7535.194106
Percentage       :          96.219233
Chain       Count              Value        Dirty Value   Percentage
Base          202        7831.276427        7535.194106    96.219233


### LiFi

In [401]:
lifi_addresses = [
    '0x1231DEB6f5749EF6cE6943a275A1D3E7486F4EaE', # LI.FI: LiFi Diamond
]
lifi_transactions = extract_all_transactions(lifi_addresses)
topic_summary(lifi_transactions)

696
5533
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 1521
0xcba69f43792f9f399347222505213b55af8e0b0b54b893085c2e27ecbe1644f1 535
0x7bfdfdb5e3a3776976e53cb0607060f54c5312701c8cba1155cc4d5394440b38 515
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 299
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 189
0x28a87b6059180e46de5fb9ab35eb043e8fe00ab45afcc7789e3934ecbbcde3ea 179
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67 166
0x38eee76fd911eabac79da7af16053e809be0e12c8637f156e77e1af309b99537 155
0xa123dc29aebf7d0c3322c8eeb5b999e859f39937950ed31056532713d0de396f 118
0x7be3e48a8a8b4d32138937e1809ac83481fffe48e49bb60e43ed1d3d50349e4c 117
0xb8543d214cab9591941648db8d40126a163bfd0db4a865678320b921e1398043 100
0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65 95
0x918554b6bd6e2895ce6553de5de0e1a69db5289aa0e4fe193a0dcd1f14347477 87
0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1 85

In [413]:
to_chain_dict = {}
lifi_transactions_filtered = {}
for transaction in lifi_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0xcba69f43792f9f399347222505213b55af8e0b0b54b893085c2e27ecbe1644f1':
                to_chain_id = int(log.data.hex()[512:576], 16)
                if to_chain_id == 1151111081099710 or to_chain_id == 20000000000001:
                    to_chain_id = 'lifi_' + str(to_chain_id)
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                lifi_transactions_filtered[transaction_hash] = [to_chain, transaction]


print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

lifi_1151111081099710 155
42161 117
56 78
8453 62
137 53
10 25
324 9
2741 9
lifi_20000000000001 8
43114 6
100 5
1088 3
81457 2
25 1
59144 1
480 1
535


In [415]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in lifi_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    if value > 0:
        print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :        4708.773903
Total Dirty Value:        4580.438603
Percentage       :          97.274550
Chain       Count              Value        Dirty Value   Percentage
Arbitrum      117        2413.307384        2384.501548    98.806375
Base           62         775.487676         762.435273    98.316878
Optimism       25         617.227528         572.215691    92.707416
BSC            78         563.038367         538.145331    95.578803
Polygon        53         226.091152         224.976198    99.506857
Solana        155          55.457554          41.117889    74.142989
Bitcoin         8          32.665125          31.866592    97.555397
ZKSync          9          13.589196          13.561039    99.792799
Avalanche       6          10.142542          10.130258    99.878892
Metis           3           1.001000           1.001000   100.000000
Abstract        9           0.486369           0.216084    44.427929
Blast           2           0.103001           0.101118   

### Wormhole

In [205]:
wormhole_addresses = [
    '0xCafd2f0A35A4459fA40C0517e17e6fA2939441CA', # Wormhole: TokenBridgeRelayer
    '0x3ee18B2214AFF97000D974cf647E7C347E8fa585', # Wormhole: Portal Token Bridge
]
wormhole_transactions = extract_all_transactions(wormhole_addresses)
topic_summary(wormhole_transactions)

375
1006
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 349
0x6eb224fb001ed210e379b335e35efe88672a8ce935d981a6896b27ffdf52a3b2 313
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 156
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 143
0xcaf280c8cfeba144da67230d9b009c8f868a75bac9a528fa0474be1ba317c169 40
0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65 4
0x1cf3b03a6cf19fa2baba4df148e9dcabedea7f8a5c07840e207e5c089be95d3e 1


In [227]:
to_chain_dict = {}
wormhole_transactions_filtered = {}
for transaction in wormhole_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0x6eb224fb001ed210e379b335e35efe88672a8ce935d981a6896b27ffdf52a3b2':
                to_chain_id = 'wormhole_' + str(int(log.data.hex()[518:522], 16))
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                wormhole_transactions_filtered[transaction_hash] = [to_chain, transaction]


print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

wormhole_1 279
wormhole_4 9
wormhole_32 4
wormhole_30 4
wormhole_21 3
wormhole_22 3
wormhole_0 3
wormhole_5 2
wormhole_16 2
wormhole_14 1
wormhole_24 1
wormhole_23 1
wormhole_8 1
313


In [233]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in wormhole_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    if value > 0:
        print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :        5406.809264
Total Dirty Value:        4552.870585
Percentage       :          84.206236
Chain       Count              Value        Dirty Value   Percentage
Solana        279        5333.217731        4480.667499    84.014337
Aptos           3          51.091000          51.048487    99.916789
Sei             4          19.898200          19.898200   100.000000
BSC             9           0.512065           0.512065   100.000000
Polygon         2           0.200268           0.199805    99.768386
Celo            1           0.200000           0.198903    99.451527
Sui             3           1.500000           0.155627    10.375118
Arbitrum        1           0.100000           0.100000   100.000000
Moonbeam        2           0.050000           0.050000   100.000000
Base            4           0.040000           0.040000   100.000000


### Synapse

In [249]:
synapse_addresses = [
    '0x7E7A0e201FD38d3ADAA9523Da6C109a07118C96a', # Synapse: Router
    '0x0000000000489d89D2B233D3375C045dfD05745F', # Synapse Protocol: RFQ Fast Bridge Router
    '0xa2569370A9D4841c9a62Fc51269110F2eB7E0171', # Synapse: Bridge Zap 1
    '0x518B553FE47208389271745155a6aBf4A961ef28', # Synapse: Bridge Zap 2
    '0x6571d6be3d8460CF5F7d6711Cd9961860029D85F', # Synapse: Bridge Zap 3
    '0x00cD000000003f7F682BE4813200893d4e690000', # Synapse Protocol: FastBridgeRouterV2
]
synapse_transactions = extract_all_transactions(synapse_addresses)
topic_summary(synapse_transactions)

367
1285
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 528
0x79c15604b92ef54d3f61f0c40caab8857927ca3d5092367163b4562c1699eb5f 258
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 174
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 130
0x120ea0364f36cdac7983bcfdd55270ca09d7f9b314a2ebc425a3b01ab1d6403a 102
0x189c623b666b1b45b83d7178f39b8c087cb09774317ca2f53c2d3c3726f222a2 86
0xda5273705dbef4bf1b902a131c2eac086b7e1476a8ab0cb4da08af1fe1bd8e3b 7


In [250]:
to_chain_dict = {}
synapse_transactions_filtered = {}
for transaction in synapse_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0x79c15604b92ef54d3f61f0c40caab8857927ca3d5092367163b4562c1699eb5f':
                to_chain_id = int(log.data.hex()[:64], 16)
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                synapse_transactions_filtered[transaction_hash] = [to_chain, transaction]
            if target_topic == '0x120ea0364f36cdac7983bcfdd55270ca09d7f9b314a2ebc425a3b01ab1d6403a':
                to_chain_id = int(log.data.hex()[64:128], 16)
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                synapse_transactions_filtered[transaction_hash] = [to_chain, transaction]

print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

42161 249
56 38
10 20
137 19
43114 14
8453 10
81457 4
1088 1
25 1
250 1
1666600000 1
1313161554 1
7700 1
360


In [252]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in synapse_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    if value > 0:
        print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :        5591.923195
Total Dirty Value:        5508.257997
Percentage       :          98.503821
Chain       Count              Value        Dirty Value   Percentage
Arbitrum      249        5056.943823        4997.846964    98.831372
Optimism       20         359.617500         359.596801    99.994244
Blast           4         100.460628         100.460628   100.000000
Base           10          43.981243          39.352488    89.475614
Avalanche      14          30.800000          10.899580    35.388248
Fantom          1           0.050000           0.050000   100.000000
Harmony         1           0.050000           0.050000   100.000000
Metis           1           0.020000           0.001536     7.678512


### deBridge

In [260]:
debridge_addresses = [
    '0xeF4fB24aD0916217251F553c0596F8Edc630EB66', # deBridge: DlnSource
    '0x663DC15D3C1aC63ff12E45Ab68FeA3F0a883C251', # deBridge: Crosschain Forwarder Proxy
]
debridge_transactions = extract_all_transactions(debridge_addresses)
topic_summary(debridge_transactions)

505
2113
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 829
0xfc8703fd57380f9dd234a89dce51333782d49c5902f307b02f03e014d18fe471 499
0xdde2f3711ab09cdddcfee16ca03e54d21fb8cf3fa647b9797913c950d38ad693 136
0x149635d19f798f6b7c74c74a500d362c89316a0ab808abe5e0c0de45da9b1d2c 134
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 129
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67 104
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 102
0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1 25
0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822 25
0xddac40937f35385a34f721af292e5a83fc5b840f722bff57c2fc71adba708c48 17
0xfec331350fce78ba658e082a71da20ac9f8d798a99b3c79681c8440cbfe77e07 10
0xac75f773e3a92f1a02b12134d65e1f47f8a14eabe4eaf1e24624918e6a8b269f 9
0xa0a19463ee116110c9b282012d9b65cc5522dc38a9520340cbaf3142e550127f 8
0xd6d4f5681c246c9f42c203e287975af1601f8df8035a9251f79aab5c8f09e2f8 8
0x095e6

In [275]:
to_chain_dict = {}
debridge_transactions_filtered = {}
for transaction in debridge_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            if len(log.topics) == 0:
                continue
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0xfc8703fd57380f9dd234a89dce51333782d49c5902f307b02f03e014d18fe471':
                to_chain_id = 'debridge_' + str(int('0x' + log.data.hex()[64*12:64*12+64], 16))
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                debridge_transactions_filtered[transaction_hash] = [to_chain, transaction]


print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

debridge_7565164 211
debridge_42161 133
debridge_56 76
debridge_8453 51
debridge_137 16
debridge_10 8
debridge_43114 2
debridge_100000002 1
debridge_100000004 1
499


In [276]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in debridge_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    if value > 0:
        print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :        2898.685351
Total Dirty Value:        2656.313769
Percentage       :          91.638569
Chain       Count              Value        Dirty Value   Percentage
Arbitrum      133        1493.381060        1390.994993    93.144009
Solana        211         769.760856         700.541224    91.007645
BSC            76         507.401620         458.756386    90.412874
Base           51          73.555636          56.327763    76.578446
Polygon        16          19.678163          18.403587    93.522890
Gnosis          1          13.704972          12.426559    90.671907
Optimism        8          11.886084          10.416088    87.632624
Avalanche       2           9.315959           8.447061    90.673017
Metis           1           0.001000           0.000107    10.722835


### Socket

In [295]:
socket_addresses = [
    '0x3a23F943181408EAC424116Af7b7790c94Cb97a5', # Socket: Gateway
]
socket_transactions = extract_all_transactions(socket_addresses)
topic_summary(socket_transactions)

727
3801
0x74594da9e31ee4068e17809037db37db496702bf7d8d63afe6f97949277d1609 748
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 612
0xa123dc29aebf7d0c3322c8eeb5b999e859f39937950ed31056532713d0de396f 469
0x6ea2964966a13d361befaca87edb26595ca75a30f3b77887d67d5a7d0e4805c0 432
0x830d2d700a97af574b186c80d40429385d24241565b08a7c559ba283a964d9b1 412
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 403
0xafc4df6845a4ab948b492800d3d8a25d538a102a2bc07cd01f1cfa097fddcff6 61
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 48
0x90890809c654f11d6e72a28fa60149770a0d11ec6c92319d6ceb2bb0a4ea1a15 47
0xb346a959ba6c0f1c7ba5426b10fd84fe4064e392a0dfcf6609e9640a0dd260d3 47
0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32 44
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67 39
0x0a0607688c86ec1775abcdbab7b33a3a35a6c9cde677c9be880150c231cc6b0b 32
0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1 24
0xd78

In [300]:
to_chain_dict = {}
socket_transactions_filtered = {}
for transaction in socket_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            if len(log.topics) == 0:
                continue
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0x74594da9e31ee4068e17809037db37db496702bf7d8d63afe6f97949277d1609':
                to_chain_id = int(log.data.hex()[128:192], 16)
                if to_chain_id == 111 or to_chain_id == 102:
                    to_chain_id = 'stargate_' + str(int('0x' + log.data.hex()[128:192], 16))
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                socket_transactions_filtered[transaction_hash] = [to_chain, transaction]

print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

8453 496
42161 94
56 60
137 46
10 27
324 19
1101 2
43114 1
stargate_111 1
stargate_102 1
57073 1
748


In [301]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in socket_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    if value > 0:
        print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :        3771.897880
Total Dirty Value:        3546.257939
Percentage       :          94.017867
Chain       Count              Value        Dirty Value   Percentage
ZKSync         19        1394.622524        1393.966873    99.952987
Base          495        1211.384957        1096.175599    90.489451
Arbitrum       89         666.770542         596.801130    89.506223
Polygon        37         265.352874         265.310007    99.983845
BSC            42         143.974254         121.057955    84.083058
Optimism       26          88.709728          71.919488    81.072830
Avalanche       1           1.000000           0.987200    98.719991
Ink             1           0.083000           0.039688    47.816498


### Symbiosis Finance

In [302]:
symbiosis_addresses = [
    '0xf621Fb08BBE51aF70e7E0F4EA63496894166Ff7F', # Symbiosis Finance: MetaRouter
]
symbiosis_transactions = extract_all_transactions(symbiosis_addresses)
topic_summary(symbiosis_transactions)

680
6517
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 2691
0x532dbb6d061eee97ab4370060f60ede10b3dc361cc1214c07ae5e34dd86e6aaf 679
0x31325fe0a1a2e6a5b1e41572156ba5b4e94f0fae7e7f63ec21e9b5ce1e4b3eab 679
0x5a297b2c9a9f94a0f4e5a796c74ad38e219d1185fccf5f79c18726a830c2b6f5 679
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 572
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67 470
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 147
0x76af224a143865a50b41496e1a73622698692c565c1214bc862f18e22d829c5e 139
0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1 71
0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822 71
0x085d06ecf4c34b237767a31c0888e121d89546a77f186f1987c6b8715e1a8caa 39
0x4d93b232a24e82b284ced7461bf4deacffe66759d5c24513e6f29e571ad78d15 38
0x19b47279256b2a23a1665c810c8d55a1758940ee09377d4f8d26497a3577dc83 35
0x103ed084e94a44c8f5f6ba8e3011507c41063177e29949083c439777d8d63f60 24
0x

In [305]:
to_chain_dict = {}
symbiosis_transactions_filtered = {}
for transaction in symbiosis_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            if len(log.topics) == 0:
                continue
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0x31325fe0a1a2e6a5b1e41572156ba5b4e94f0fae7e7f63ec21e9b5ce1e4b3eab':
                to_chain_id = int(log.topics[2].hex(), 16)
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                symbiosis_transactions_filtered[transaction_hash] = [to_chain, transaction]


print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

56288 679
679


In [306]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in symbiosis_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    if value > 0:
        print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :        2915.591808
Total Dirty Value:        2797.070942
Percentage       :          95.934929
Chain       Count              Value        Dirty Value   Percentage
Boba_BNB      679        2915.591808        2797.070942    95.934929


### SWFT Swap

In [307]:
swft_addresses = [
    '0x92e929d8B2c8430BcAF4cD87654789578BB2b786'
]
swft_transactions = extract_all_transactions(swft_addresses)
topic_summary(swft_transactions)

710
738
0x4e96fb90a89341a56db7ad2bbf04c715bbf20be6a9a9e764671f718c4697649a 316
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 211
0x45f377f845e1cc76ae2c08f990e15d58bcb732db46f92a4852b956580c3a162f 211


In [348]:
to_chain_dict = {}
swft_transactions_filtered = {}
for transaction in swft_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            if len(log.topics) == 0:
                continue
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0x45f377f845e1cc76ae2c08f990e15d58bcb732db46f92a4852b956580c3a162f' or \
                target_topic == '0x4e96fb90a89341a56db7ad2bbf04c715bbf20be6a9a9e764671f718c4697649a':
                info = bytes.fromhex(log.data.hex()[384:]).decode('ascii').replace('\x00', '')
                to_chain_id = re.findall(r'\((.*?)\)', info)
                # to_chain_id = ''.join(to_chain_id)
                if to_chain_id:
                    to_chain_id = to_chain_id[0]
                else:
                    to_chain_id = re.match(r'^[a-zA-Z0-9]+', info)
                    if to_chain_id:
                        to_chain_id = to_chain_id[0]
                    else:
                        continue
                to_chain_id = 'swft_' + to_chain_id
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                swft_transactions_filtered[transaction_hash] = [to_chain, transaction]


print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)


swft_TRON 336
swft_BSC 72
swft_TRX 39
swft_ARB 23
swft_MATIC 10
swft_ERC20 10
swft_SOL 9
swft_C-Chain 8
swft_OKT 3
swft_LINK 3
swft_MNT 2
swft_Optimism 1
swft_ZKSYNC 1
swft_BASE 1
swft_TON 1


In [349]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in swft_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    if value > 0:
        print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :        1721.814145
Total Dirty Value:        1697.464226
Percentage       :          98.585799
Chain       Count              Value        Dirty Value   Percentage
TRON          375        1064.161379        1052.855953    98.937621
BSC            72         344.358242         331.380024    96.231187
Arbitrum       23         115.232310         115.167266    99.943554
Solana          9          61.172210          61.172160    99.999919
Avalanche       8          54.055579          54.055559    99.999964
Ethereum       13          51.113962          51.113934    99.999945
Polygon        10          30.310155          30.309021    99.996259
ZKSync          1           0.985605           0.985605   100.000000
OKXChain        3           0.181678           0.181678   100.000000
Mantle          2           0.120000           0.120000   100.000000
Base            1           0.087025           0.087025   100.000000
TON             1           0.020000           0.020000   

### Transit Swap

In [350]:
transit_addresses = [
    '0xb45A2DDA996C32E93B8c47098E90Ed0E7ab18E39', # Transit Swap: Rounter V4 2
    '0x00000047bB99ea4D791bb749D970DE71EE0b1A34', # Transit Swap: Rounter V5
]
transit_transactions = extract_all_transactions(transit_addresses)
topic_summary(transit_transactions)

869
4235
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 1382
0x2251435bd151cd72851a82be055bf6d1c3d7f34d08d56493dddf874229b8e897 507
0x4e96fb90a89341a56db7ad2bbf04c715bbf20be6a9a9e764671f718c4697649a 469
0x7055e3d08e2c20429c6b162f3e3bee3f426d59896e66084c3580dc353e54129d 362
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 230
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67 201
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 199
0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1 140
0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822 140
0x45f377f845e1cc76ae2c08f990e15d58bcb732db46f92a4852b956580c3a162f 115
0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65 60
0x7063ee7ac21ca792eb7d62d3a65598a5c986c4b0f7bd701aa453eb8a1387c956 60
0x3d665fb91a05a4c30e7e5da7d4eb00bda6b3a5f9b69ffc766b80912c4438db19 44
0xfe2e49614659d9447156c9e3846112ea8edda361dcd696be486f6f977ce854e5 41


In [378]:
to_chain_dict = {}
transit_transactions_filtered = {}
for transaction in transit_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            if len(log.topics) == 0:
                continue
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0x2251435bd151cd72851a82be055bf6d1c3d7f34d08d56493dddf874229b8e897':
                to_chain_id = 'transit_' + str(int(log.data.hex()[128:192], 16))
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                transit_transactions_filtered[transaction_hash] = [to_chain, transaction]
                
            if target_topic == '0x7055e3d08e2c20429c6b162f3e3bee3f426d59896e66084c3580dc353e54129d':
                to_chain_id = 'transit_' + str(int(log.data.hex()[384:384+64], 16))
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                transit_transactions_filtered[transaction_hash] = [to_chain, transaction]


print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))
            

transit_95500 484
transit_0 193
transit_56 76
transit_42161 41
transit_728126428 39
transit_10 22
transit_137 6
transit_128 3
transit_8453 2
transit_250 1
transit_324 1
transit_185 1
869


In [380]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in transit_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    if value > 0:
        print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :        3908.200608
Total Dirty Value:        3874.549176
Percentage       :          99.138953
Chain       Count              Value        Dirty Value   Percentage
TRON          523        2508.637335        2500.319849    99.668446
Ethereum      193        1045.832546        1032.115096    98.688370
BSC            76         140.791733         130.486084    92.680217
Arbitrum       41         120.636509         119.763730    99.276522
Optimism       22          79.319514          79.319513    99.999999
Polygon         6          12.409969          12.409683    99.997700
Fantom          1           0.100000           0.100000   100.000000
Base            2           0.460000           0.033295     7.237960
ZKSync          1           0.012000           0.001864    15.536847
Mint            1           0.001002           0.000062     6.194591


### Celer Network

In [387]:
celer_addresses = [
    '0x5427FEFA711Eff984124bFBB1AB6fbf5E3DA1820', # Celer Network: cBridge V2
    # '0x841ce48F9446C8E281D3F1444cB859b4A6D0738C', # Celer Network: cBridge
    # '0x9a66644084108a1bC23A9cCd50d6d63E53098dB6',
    # '0x98E9D288743839e96A8005a6B51C770Bbf7788C0',
    # '0xd10c833f4305E1053a64Bc738c550381f48104Ca',
    # '0xbfa2F68bf9Ad60Dc3cFB1cEF04730Eb7FA251424',
]
celer_transactions = extract_all_transactions(celer_addresses)
topic_summary(celer_transactions)

203
407
0x89d8051e597ab4178a863a5190407b98abfeff406aa8db90c59af76612e58f01 197
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 116
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 87
0x48a1ab26f3aa7b62bb6b6e8eed182f292b84eb7b006c0254386b268af20774be 6
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 1


In [390]:
to_chain_dict = {}
celer_transactions_filtered = {}
for transaction in celer_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            if len(log.topics) == 0:
                continue
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0x89d8051e597ab4178a863a5190407b98abfeff406aa8db90c59af76612e58f01':
                to_chain_id = int(log.data.hex()[-192:-128], 16)
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                celer_transactions_filtered[transaction_hash] = [to_chain, transaction]


print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

56 92
10 67
42161 18
137 11
250 4
1101 2
1088 1
43114 1
324 1
197


In [391]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in celer_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    if value > 0:
        print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :        2056.578995
Total Dirty Value:        2037.132156
Percentage       :          99.054408
Chain       Count              Value        Dirty Value   Percentage
BSC            92         999.603888         981.082126    98.147090
Optimism       67         440.280952         440.233655    99.989258
Arbitrum       18         369.954155         369.893623    99.983638
Polygon        13         245.720000         245.637309    99.966347
Fantom          4           1.000000           0.283508    28.350753
ZKSync          1           0.020000           0.001935     9.675780


### Hop Protocol

In [392]:
hop_addresses = [
    '0xb8901acB165ed027E32754E0FFe830802919727f', # 0xb8901acB165ed027E32754E0FFe830802919727f
]
hop_transactions = extract_all_transactions(hop_addresses)
topic_summary(hop_transactions)

201
626
0x0a0607688c86ec1775abcdbab7b33a3a35a6c9cde677c9be880150c231cc6b0b 201
0xff64905f73a67fb594e0f940a8075a860db489ad991e032f48c81123eb52d60b 131
0x5e3c1311ea442664e8b1611bfabef659120ea7a0a2cfc0667700bebc69cbffe1 126
0xcb0f7ffd78f9aee47a248fae8db181db6eee833039123e026dcbff529522e52a 58
0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32 34
0x8ebb2ec2465bdb2a06a66fc37a0963af8a2a6a1479d81d56fdb8cbb98096d546 34
0x4b388aecf9fa6cc92253704e5975a6129a4f735bdbd99567df4ed0094ee4ceb5 24
0x103fed9db65eac19c4d870f49ab7520fe03b99f1838e5996caf47e9e43308392 10
0x23be8e12e420b5da9fb98d8102572f640fb3c11a0085060472dfc0ed194b3cf7 5
0x482515ce3d9494a37ce83f18b72b363449458435fafdd7a53ddea7460fe01b58 1
0xea3b023b4c8680d4b4824f0143132c95476359a2bb70a81d6c5a36f6918f6339 1
0xe856c2b8bd4eb0027ce32eeaf595c21b0b6b4644b326e5b7bd80a1cf8db72e6c 1


In [399]:
to_chain_dict = {}
hop_transactions_filtered = {}
for transaction in hop_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            if len(log.topics) == 0:
                continue
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0x0a0607688c86ec1775abcdbab7b33a3a35a6c9cde677c9be880150c231cc6b0b':
                to_chain_id = int(log.topics[1].hex(), 16)
                if to_chain_id not in to_chain_dict:
                    to_chain_dict[to_chain_id] = 1
                else:
                    to_chain_dict[to_chain_id] += 1
                to_chain = CHAIN_ID.get(to_chain_id, 'Unknown')
                hop_transactions_filtered[transaction_hash] = [to_chain, transaction]


print('='*50)
to_chain_dict = dict(sorted(to_chain_dict.items(), key=lambda item: item[1], reverse=True))
for to_chain_id, count in to_chain_dict.items():
    print(to_chain_id, count)
print('='*50)
print(sum(to_chain_dict.values()))

42161 131
10 37
8453 21
137 10
100 1
59144 1
201


In [400]:
total_value = 0
total_dirty_value = 0
summary = {}
for transaction_hash, [to_chain, transaction] in hop_transactions_filtered.items():
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    if to_chain not in summary:
        summary[to_chain] = (1, transaction[8], transaction[9])
    else:
        summary[to_chain] = (summary[to_chain][0] + 1, summary[to_chain][1] + transaction[8], summary[to_chain][2] + transaction[9])

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
summary = dict(sorted(summary.items(), key=lambda item: item[1][2], reverse=True))
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
for chain, (count, value, dirty_value) in summary.items():
    if value > 0:
        print(f"{chain:<10} {count:6} {int(value)/1e18:18.6f} {int(dirty_value)/1e18:18.6f} {dirty_value/value*100:12.6f}")

Total Value      :        1706.417208
Total Dirty Value:        1633.281025
Percentage       :          95.714050
Chain       Count              Value        Dirty Value   Percentage
Arbitrum      131        1068.377680        1008.515009    94.396863
Optimism       37         426.381635         415.218892    97.381983
Base           21         168.219836         167.520614    99.584341
Polygon        10          32.451057          31.936419    98.414109
Gnosis          1          10.000000           9.989868    99.898681
Linea           1           0.987000           0.100222    10.154235


### Polygon

In [528]:
polygon_addresses = [
    '0xA0c68C638235ee32657e8f720a23ceC1bFc77C77', # Polygon (Matic): Bridge
]
polygon_transactions = extract_all_transactions(polygon_addresses)
topic_summary(polygon_transactions)

182
330
0x103fed9db65eac19c4d870f49ab7520fe03b99f1838e5996caf47e9e43308392 106
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 97
0x3e799b2d61372379e767ef8f04d65089179b7a6f63f9be3065806456c7309f1b 70
0x9b217a401a5ddf7c4d474074aff9958a18d48690d77cc2151c4706aa7348b401 35
0x0fc0eed41f72d3da77d0f53b9594fc7073acd15ee9d7c536819a70a67c57ef3c 15
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 3
0xbb61bd1b26b3684c7c028ff1a8f6dabcac2fac8ac57b66fa6b1efb6edeab03c4 3
0x5345c2beb0e49c805f42bb70c4ec5c3c3d9680ce45b8f4529c028d5f3e0f7a0d 1


In [529]:
total_value = 0
total_dirty_value = 0
for transaction in polygon_transactions:
    total_value += transaction[8]
    total_dirty_value += transaction[9]

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

print('='*50)
print(f"{'Chain':<10} {'Count':>6} {'Value':>18} {'Dirty Value':>18} {'Percentage':>12}")
print(f"{'Polygon':<10} {len(arbitrum_transactions):6} {int(total_value)/1e18:18.6f} {int(total_dirty_value)/1e18:18.6f} {total_dirty_value/total_value*100:12.6f}")

Total Value      :        1547.708986
Total Dirty Value:        1540.991210
Percentage       :          99.565953
Chain       Count              Value        Dirty Value   Percentage
Polygon      1673        1547.708986        1540.991210    99.565953


### Chainflip

In [8]:
def extract_smart_contract_transactions():
    with psycopg2.connect(**CONN_PARAMS) as conn:
        with conn.cursor() as curs:
            transactions = []
            sql_select = \
            """
            SELECT * FROM related_transactions_final as T
            JOIN related_addresses_final as A
            ON T.to_address = A.address
            WHERE A.is_smart_contract = TRUE 
            ORDER BY block_number ASC; 
            """
            curs.execute(sql_select)
            rows = curs.fetchall()
            for row in rows:
                transactions.append(row)
            return transactions

In [9]:
all_smart_contract_transactions = extract_smart_contract_transactions()
print(f"Total Smart Contract Transactions: {len(all_smart_contract_transactions)}")

Total Smart Contract Transactions: 485607


In [10]:
chainflip_transactions = []
for transaction in all_smart_contract_transactions:
    transaction_hash = transaction[3]
    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    flag = False
    if transaction_receipt:
        for log in transaction_receipt.logs:
            if len(log.topics) == 0:
                continue
            target_address = w3.to_checksum_address(log.address)
            # print(target_address)
            if target_address == '0xF5e10380213880111522dd0efD3dbb45b9f62Bcc':
                flag = True
                break
    if flag:
        chainflip_transactions.append(transaction)

In [11]:
len(chainflip_transactions)
topic_summary(chainflip_transactions)

340
0xa5df10d7a7c736bd7231e790267747aebface82ad6ee778d26a4aa99eb1c2f7b 284
0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef 25
0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c 6
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67 5
0x834b524d9f8ccbd31b00b671c896697b96eb4398c0f56e9386a21f5df61e3ce3 4
0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925 3
0x61a4ff3bb8bbf861e1aa28d574e4b88959510ac2ae1b845521bb8c13c33d566b 2
0x4d93b232a24e82b284ced7461bf4deacffe66759d5c24513e6f29e571ad78d15 2
0xdf4363408b2d9811d1e5c23efdb5bae0b7a68bd9de2de1cbae18a11be3e67ef5 1
0xf14fbd8b6e3ad3ae34babfa1f3b6a099f57643662f4cfc24eb335ae8718f534b 1
0x012c155f3836c4edb9222305b909a109f9efa46288efffe40a0e66da3a9a9800 1
0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1 1
0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822 1
0x6757034e841d88190e95fa8a7570ce6cb721257474400c3282ec536eab810540 1
0xdc004dbca4ef9c966218431ee

In [12]:
total_value = 0
total_dirty_value = 0
for transaction in chainflip_transactions:
    transaction_hash = transaction[3]

    transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
    if transaction_receipt:
        for log in transaction_receipt.logs:
            if len(log.topics) == 0:
                continue
            target_topic = '0x' + log.topics[0].hex()
            if target_topic == '0xa5df10d7a7c736bd7231e790267747aebface82ad6ee778d26a4aa99eb1c2f7b':
                print(transaction_hash)
                
    total_value += transaction[8]
    total_dirty_value += transaction[9]

    

print('='*50)
print(f'Total Value      : {int(total_value)/1e18:18.6f}')
print(f'Total Dirty Value: {int(total_dirty_value)/1e18:18.6f}')
print(f'Percentage       : {total_dirty_value/total_value*100:18.6f}')

0xae545d0d43fcb078d829d74d1f8c537a448cc2810e68b080535517c903b0b390
0x6a448d323fa85dfdb381599c4ed64c29af2b8415b687cf9371221018f3cefb3f
0xbc91465d9bac9e0054962ee44432720b7a0551c9c3c7177be836055f6a94a552
0xcea986acdba23928c07e694f4c2e485fe2b225035987c19a84c9deb7eece2e3b
0x3070a29e097a1a8db6af18036e7dd1f9f25b245f9bdc3f29a2f817e774bae5f0
0x2f24d0ccd8964c9e31d357f346db0aedec3bb34d017af94047bb80296e8a37a2
0x60f848cbe58af1ad51cbc782033cc9ca8bc92e3b12e20b6a5dfca92da4fd3eb4
0xf1303c4dbeed909de4b22b4e71b43196f09a465c06b8597d9a70fbc62287795c
0x3d844645f6e26df9d111dc4c0429f69f5ee2ff6c9ff94dcb6af9a66f3a2d2840
0xfd3f0664da7c475ee80488da6600032bc7fdc970001b24e187584b8de073584d
0xd4cbc48939fc0d69c28b1341f3f8470e5dba16a1ae1461ef79f141d771f91ee5
0xb1080bcd7c333ab30705bea311cfa1fcd5910249e9941f0e145dff72076d62f9
0x631f44710e0e8511dc642fc904707691168b3a11712e012e0cce2981f0fc7e7b
0xf2f6be2d68b1324e23ff35d1c78a0240d8d92dc215fe27d05799da77e204a41b
0xda15ab487202039ce93818ef1d730bf8b9208e9133a025f39174b7d39d15

## Sankey Diagram

In [4]:
# Define the nodes
nodes = [
    "THORChain", # 0
    "Arbitrum", # 1
    "OKX", # 2
    "Across", # 3
    "Stargate", # 4
    "Optimism", # 5
    "Mayan", # 6
    "Multichain", # 7
    "ZKSync", # 8
    "Base", # 9
    "LiFi", # 10
    "Wormhole", # 11 
    "Synapse", # 12
    "deBridge", # 13
    "Socket", # 14
    "Symbiosis", # 15
    "SWFT", # 16
    "Transit", # 17
    "Celer", # 18
    "Hop",  # 19
    "Polygon", # 20
    
    "Bitcoin", # 21
    "Litecoin", # 22
    "Avalanche", # 23
    "BSC", # 24
    "Thorchain", # 25
    "Arbitrum", # 26
    "Tron", # 27
    "Base", # 28 
    "Optimism", # 29
    "Sui", # 30
    "Polygon", # 31
    "Solana", # 32
    "Linea", # 33
    "ZKSync", # 34
    "Boba_BNB", # 35
    "Other"
]


# Indices of the source nodes
source = [
     0,  0,  0,  0,
     1,
     2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
     3,  3,  3,  3,  3,  3,
     4,  4,  4,  4,
     5, 
     6,  6,  6,
     7,  7,  7,  7,
     8, 
     9, 
    10, 10, 10, 10, 10, 10, 10, 10, 10,
    11, 11, 11, 11, 11, 11,
    12, 12, 12, 12,
    13, 13, 13, 13, 13, 13, 13,
    14, 14, 14, 14, 14, 14, 14,
    15, 
    16, 16, 16, 16, 16, 16, 16,
    17, 17, 17, 17, 17,
    18, 18, 18, 18,
    19, 19, 19, 19,
    20,
]
# Indices of the target nodes
target = [
    21, 22, 23, 24, 
    26,
    27, 21, 26, 28, 29, 30, 31, 32, 24, 34,
    26, 29, 28, 33, 31, 34,
    26, 29, 28, 33,
    29,
    32, 26, 24,
    31, 24, 29, 26,
    34,
    28,
    26, 28, 29, 24, 31, 32, 21, 34, 23,
    32, 24, 31, 30, 26, 28,
    26, 29, 28, 23,
    26, 32, 24, 28, 31, 29, 23,
    34, 28, 26, 31, 24, 29, 23,
    35,
    27, 24, 26, 32, 23, 31, 34,
    27, 24, 26, 29, 31,
    24, 29, 26, 31,
    26, 29, 28, 31,
    31,
]
# Magnitudes of the flows
value = [
    112176.348190, 8802.519171, 908.302238, 707.002416,
    62610.416893,
    11444.552409, 4698.071371, 1771.070669, 910.239124, 511.879340, 456.146404, 316.200305, 234.116773, 77.286655, 1.285143,
    17456.057804, 3894.022346, 2851.950005, 1376.538517, 341.376623, 234.917929,
    18015.768350, 5481.918716, 2075.592858, 850.496897,
    12536.994640,
    9566.949437, 56.000000, 8.352641,
    4413.580886, 3679.143381, 1407.159880, 1119.727000,
    12006.504170,
    7535.194106,
    2384.501548, 762.435273, 572.215691, 538.145331, 224.976198, 41.117889, 31.866592, 13.561039, 10.130258,
    4480.667499, 0.512065, 0.199805,  0.155627, 0.100000, 0.040000,
    4997.846964, 359.596801, 39.352488, 10.899580,
    1390.994993, 700.541224, 458.756386, 56.327763, 18.403587, 10.416088, 8.447061,
    1393.966873, 1096.175599,  596.801130, 265.310007, 121.057955, 71.919488, 0.987200,
    2797.070942,
    1052.855953, 331.380024, 115.167266, 61.172160, 54.055559, 30.309021, 0.985605,
    2500.319849, 130.486084, 119.763730, 79.319513, 12.409683,
    981.082126, 440.233655, 369.893623, 245.637309,
    1008.515009, 415.218892, 167.520614, 31.936419,
    1540.991210,
]   

print(f'{sum(value):.6f}')

# Calculate the total flow to each destination
data = pd.DataFrame({"source": source, "target": target, "value": value})
# total_flows = data.groupby("target")["value"].sum()

# # Determine which destinations need to be aggregated
# aggregated_target = len(nodes) - 1  # Index for the "Other" node
# filtered_links = {"source": [], "target": [], "value": []}

# # Create new links with aggregation logic
# for i, row in data.iterrows():
#     if total_flows[row["target"]] < 1000:  # Sum of flows to the destination < 1000
#         filtered_links["source"].append(row["source"])
#         filtered_links["target"].append(aggregated_target)
#         filtered_links["value"].append(row["value"])
#     else:
#         filtered_links["source"].append(row["source"])
#         filtered_links["target"].append(row["target"])
#         filtered_links["value"].append(row["value"])


# df_grouped = data.groupby('target', as_index=False).sum()
# for i, row in df_grouped.iterrows():
#     print(row)

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    arrangement = 'perpendicular',
    node=dict(
        pad=10,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes,
    ),
    link=dict(
        source=source,
        target=target,
        value=value,),
)])

fig.update_layout(
    font=dict(size=16, color="black"),
    width=500,  # Set the width of the plot
    height=700,  # Set the height of the plot
    margin=dict(l=0, r=0, t=0, b=0)
)
fig.show()

343678.433535


In [20]:
data[data['target'] == 21]['value'].sum()
data[data['target'].isin([23, 24, 26, 27, 28, 29, 31, 33, 34, 35])]['value'].sum()
data[data['target'] == 22]['value'].sum()
data[data['target'] == 32]['value'].sum()

np.float64(15084.564981999998)